In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)

(2838, 370) (939, 369)


In [3]:
train.head(2)

,Year,Field ID,Field Area,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,...,Day 358,Day 359,Day 360,Day 361,Day 362,Day 363,Day 364,Day 365,Day 366,Culture
0,2018,517,48.64,0.5369,0.5371,0.5375,0.5379,0.5384,0.5390,0.5397,...,0.4362,0.4322,0.4283,0.4243,0.4203,0.4163,0.4123,0.4084,NaN,4
1,2019,1395,80.41,0.1159,0.1096,0.1034,0.0971,0.0909,0.0846,0.0783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [4]:
X_train = train.drop(['Year','Field ID','Field Area','Culture'], axis=1)
y_train = train['Culture']

X_test = test.drop(['Year','Field ID','Field Area'],axis=1)
X_train.shape, X_test.shape, y_train.shape

((2838, 366), (939, 366), (2838,))

In [5]:
%%time
imputer = KNNImputer(n_neighbors=9, weights="distance")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

Wall time: 13.1 s


In [9]:
X_tt, X_tv, y_tt, y_tv = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [10]:
model = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=15, max_features = 'sqrt')
model.fit(X_tt, y_tt)
preds_valid = model.predict(X_tv)

print(f1_score(y_tv, preds_valid, average='weighted'))

0.8825085725745289


In [12]:
model = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=15, max_features = 'sqrt')
model.fit(X_train, y_train)
preds_valid = model.predict(X_tv)

print(f1_score(y_tv, preds_valid, average='weighted'))

0.998065063420677


In [13]:
preds = model.predict(X_test)

pd.Series(preds).to_csv('preds.csv', index=False, header=['Culture'])